In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://hospital1-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


In [4]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, "This is a response from Bob"))
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener, message_listener], defaults=True)

## Copy Invite from Researcher

In [5]:
#Paste in invitation from researcher agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '31d22c96-b531-467a-8637-867f0e9199ac', 'label': 'Health Researcher', 'serviceEndpoint': 'http://172.17.0.1:8040', 'recipientKeys': ['Hu8va1kwBxycxKa731q6FeenAmhk5s3x2X3MXXR7CckE']}

In [6]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


Connection Handler Called
Connection ec35bfc5-68ef-45c4-87c6-5d9f63819c77 in State invitation
Connection Handler Called
Connection ec35bfc5-68ef-45c4-87c6-5d9f63819c77 in State request
Connection {'invitation_key': 'Hu8va1kwBxycxKa731q6FeenAmhk5s3x2X3MXXR7CckE', 'state': 'request', 'connection_id': 'ec35bfc5-68ef-45c4-87c6-5d9f63819c77', 'routing_state': 'none', 'updated_at': '2021-01-23 14:22:15.871577Z', 'initiator': 'external', 'invitation_mode': 'once', 'created_at': '2021-01-23 14:22:15.851490Z', 'accept': 'manual', 'their_label': 'Health Researcher', 'my_did': 'ALrPXCqEuqa5wYFUDDkfeo', 'request_id': 'b9560178-dc00-4496-9a8c-2da146166b4d'}
Connection Handler Called
Connection ec35bfc5-68ef-45c4-87c6-5d9f63819c77 in State response
Connection Handler Called
Connection ec35bfc5-68ef-45c4-87c6-5d9f63819c77 in State active


In [ ]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

In [ ]:
response = await agent_controller.terminate()
print(response)